In [16]:
addprocs(11)

11-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

In [17]:
@everywhere using FunctionalCollections
@everywhere using Iterators
@everywhere using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")

6-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util/"                                                       
 "."                                                              
 "../util/"                                                       

In [18]:
@everywhere const START_MARKER1 = "**START1**"
@everywhere const START_MARKER2 = "**START2**"
@everywhere const END_MARKER1 = "**END1**"
@everywhere const END_MARKER2 = "**END2**"

@everywhere atis_data = open("atis_data2.jsz","r") do fh
    deserialize(fh)    
end


In [19]:
@everywhere using PyCall
#http://www.nltk.org/howto/probability.html
@everywhere @pyimport nltk
@everywhere  @pyimport nltk.probability as nltk_prob


@everywhere kn_prob_dist = nltk_prob.KneserNeyProbDist(pycall(nltk_prob.FreqDist, PyObject, atis_data["trigrams"]))
@everywhere function trigram_model(given1::AbstractString, given2::AbstractString, event::AbstractString)
    kn_prob_dist[:prob]((given1, given2, event))
end

/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/draw/__init__.py:15: UserWarning: nltk.draw package not loaded (please install Tkinter library).
  warnings.warn("nltk.draw package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/draw/__init__.py:15: Us

In [20]:
@everywhere typealias S ASCIIString
@everywhere typealias State{T} Tuple{T,T}

@everywhere function transition_prob(cur_state::State{S}, next_word::S)
    trigram_model(cur_state..., next_word)
end


@everywhere _get_options_cache_type = Dict{Tuple{State{S}, Any}, Vector{Tuple{plist{S}, Float64}}}
@everywhere _get_options_caches = Dict{Vector{S}, _get_options_cache_type}()

#"""
#returns all possible orderings of the remaining words.
#the freewords are also inserted into every possible position, EXCEPT at the end.
#Freewords also will not be inserted after other freewords
#(For now)
#"""
@everywhere function get_all_orders(free_words::Vector{S}, unordered_words::Vector{S}; ϵ=0.0)
    _get_options_cache = get!(_get_options_cache_type, _get_options_caches, free_words)
    #different cache if different freewords
    
    function get_options(cur_state::State{S})
        #Can never End in a freeword
        tp = transition_prob(cur_state, END_MARKER1)
        # Given P(END_MARKER2 | curstatep[2]==END_MARKER1) = 1.0
        # Do not need to consider P(END_MARKER2 | curstatep[2]==END_MARKER1, curstatep[1])
        [
            (EmptyList{S}(), tp)
        ]
    end
    
    function get_options(state::State{S}, remaining_words::S...)
        get!(_get_options_cache, (state, remaining_words)) do
            _get_options(state, remaining_words...)
        end
    end

    function _get_options_inner(cur_state, word, remaining_words, get_option_fun)
        tp = transition_prob(cur_state, word) 
        next_state = (cur_state[2], word)
        
        if tp<=ϵ #Stop early if probability is too low
            return
        end

        for (tail, next_tp) in get_option_fun(next_state, remaining_words...)
            produce(cons(word, tail),tp*next_tp)
        end
    end

    function _get_options_nofree(cur_state::State{S}, remaining_words::S...)
        (@task begin        
            for ii in 1:length(remaining_words)
                word = remaining_words[ii]
                new_remaining_words = remaining_words[[1:ii-1; ii+1:end]]
                
                _get_options_inner(cur_state, word, new_remaining_words, get_options)
            end
        end)
    end
    
    function _get_options_free(cur_state::State{S}, remaining_words::S...)
        @task begin        
            for word in free_words
                #Important: get_options_free, only chains to _get_options_nofree
                _get_options_inner(cur_state, word, remaining_words, _get_options_nofree)
            end
        end
    end
    
    function _get_options(cur_state::State{S}, remaining_words::S...)
        chain(_get_options_nofree(cur_state, remaining_words...),
             _get_options_free(cur_state, remaining_words...)) |> collect
    end
   
    initial_state = (START_MARKER1, START_MARKER2)
    get_options(initial_state, unordered_words...)
end

In [21]:
@everywhere function order(unordered_words::Vector{S}, free_words=S[]::Vector{S}; ϵ=0.0::Float64, best_n=1::Int)
    orders_and_probs = get_all_orders(free_words, unordered_words, ϵ=0.0)
    best_n = min(best_n, length(orders_and_probs))
    if best_n==1 #4x as fast as the else
        max_prob, max_ii = @pipe orders_and_probs |> map(op->op[2],_) |> findmax
        orders_and_probs[max_ii], max_prob
    else
        select!(orders_and_probs, 1:best_n, by=op->-op[2] )
    end
end

@everywhere function order(unordered_words::Vector{S},free_words=S[]::Vector{S}, ϵ=0.0,best_n=1::Int)
    order(unordered_words,free_words; ϵ=0.0, best_n=best_n)[1][1]
end

In [22]:

unordered_output, test_set = open("atis_res_glove.jsz","r") do fh
    deserialize(fh)    
end

#data = open("atis_data2.jsz","r") do fh
#    deserialize(fh)    
#end

#const zeroed_words = data["zeroed_words"]
@everywhere zeroed_words = ASCIIString[]

In [23]:
short_cases = [Bool[length(ws) <=10 for ws in unordered_output]]


13038-element Array{Bool,1}:
 false
  true
  true
  true
  true
  true
  true
  true
  true
  true
 false
  true
  true
     ⋮
 false
  true
  true
 false
 false
 false
 false
  true
 false
 false
 false
  true

 in depwarn at deprecated.jl:73
 in oldstyle_vcat_warning at ./abstractarray.jl:29
 in anonymous at no file
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:177
 in eventloop at /home/ubuntu/.julia/v0.5/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:443
while loading In[23], in expression starting on line 1


In [24]:

eval_cases = [1:100]
true_ordered_sents = test_set[short_cases][eval_cases]
ordered_sents = pmap(unordered_output[short_cases][eval_cases]) do unordered_words
    order(unordered_words, zeroed_words)
end


100-element Array{Any,1}:
 ASCIIString(i, want, to, go, from, denver, to, oakland)                        
 ASCIIString(what, is, the, cheapest, flight, from, denver, to, oakland)        
 ASCIIString(one, way, please)                                                  
 ASCIIString(i, want, to, leave, after, five, p, m)                             
 ASCIIString(sure, i, want, to, go, from, philadelphia, to, dallas)             
 ASCIIString(on, which, flights, will, lunch, be, served)                       
 ASCIIString(which, flights, serve, lunch)                                      
 ASCIIString(thank, you)                                                        
 ASCIIString(i, live, in, denver)                                               
 ASCIIString(what, is, the, least, expensive, flight, from, atlanta, to, denver)
 ASCIIString(is, there, a, round, trip, flight, that, stops, in, pittsburgh)    
 ASCIIString(round, trip, please)                                               
 A

In [76]:
perfect_matches = Bool[]  
for ii in 1:length(ordered_sents)
    ordered_words = ordered_sents[ii]
    actual_words = true_ordered_sents[ii]
    
    match = ordered_words == actual_words
    push!(perfect_matches, match)
    println("$ii - $match")
end
mean(perfect_matches)

1 - true


0.86

2 - true
3 - true
4 - true
5 - true
6 - true
7 - true
8 - true
9 - true
10 - false
11 - true
12 - true
13 - true
14 - true
15 - true
16 - true
17 - true
18 - true
19 - false
20 - true
21 - true
22 - true
23 - true
24 - true
25 - true
26 - true
27 - true
28 - true
29 - true
30 - true
31 - true
32 - true
33 - true
34 - true
35 - true
36 - true
37 - true
38 - true
39 - true
40 - true
41 - true
42 - false
43 - true
44 - true
45 - false
46 - true
47 - true
48 - true
49 - true
50 - false
51 - true
52 - true
53 - true
54 - true
55 - true
56 - true
57 - true
58 - true
59 - true
60 - true
61 - true
62 - false
63 - true
64 - true
65 - true
66 - true
67 - true
68 - true
69 - true
70 - true
71 - true
72 - true
73 - false
74 - true
75 - true
76 - true
77 - true
78 - true
79 - true
80 - true
81 - true
82 - true
83 - false
84 - false
85 - true
86 - false
87 - true
88 - false
89 - false
90 - false
91 - true
92 - true
93 - true
94 - true
95 - true
96 - true
97 - true
98 - true
99 - false
100 - true


In [87]:
@pyimport nltk
@pyimport nltk.translate.bleu_score as nltk_bleu

function bleu_score(candidate, reference)
    reference = reference |> collect
    candidate = candidate |> collect
    
    if reference==candidate
    weights = [1,1,1,1]/4
    nltk_bleu.bleu(Any[reference],candidate, weights)
end

bleu_score (generic function with 1 method)

In [109]:
true_ordered_sents[3] == ordered_sents[3]

true

In [105]:
map(bleu_score, ordered_sents[1:40],true_ordered_sents[1:40])


40-element Array{Real,1}:
 1.0     
 1.0     
 0       
 1.0     
 1.0     
 1.0     
 1.0     
 0       
 1.0     
 0.698534
 1.0     
 0       
 1.0     
 ⋮       
 1.0     
 1.0     
 1.0     
 1.0     
 1.0     
 1.0     
 1.0     
 1.0     
 1.0     
 1.0     
 1.0     
 1.0     

In [108]:
bleu_score(true_ordered_sents[3], true_ordered_sents[3])

0

In [107]:
true_ordered_sents

100-element Array{Array{SubString{ASCIIString},1},1}:
 SubString{ASCIIString}["i","want","to","go","from","denver","to","oakland"]                          
 SubString{ASCIIString}["what","is","the","cheapest","flight","from","denver","to","oakland"]         
 SubString{ASCIIString}["one","way","please"]                                                         
 SubString{ASCIIString}["i","want","to","leave","after","five","p","m"]                               
 SubString{ASCIIString}["sure","i","want","to","go","from","philadelphia","to","dallas"]              
 SubString{ASCIIString}["on","which","flights","will","lunch","be","served"]                          
 SubString{ASCIIString}["which","flights","serve","lunch"]                                            
 SubString{ASCIIString}["thank","you"]                                                                
 SubString{ASCIIString}["i","live","in","denver"]                                                     
 SubString{ASCIIStr

In [106]:
ordered_sents

100-element Array{Any,1}:
 ASCIIString(i, want, to, go, from, denver, to, oakland)                        
 ASCIIString(what, is, the, cheapest, flight, from, denver, to, oakland)        
 ASCIIString(one, way, please)                                                  
 ASCIIString(i, want, to, leave, after, five, p, m)                             
 ASCIIString(sure, i, want, to, go, from, philadelphia, to, dallas)             
 ASCIIString(on, which, flights, will, lunch, be, served)                       
 ASCIIString(which, flights, serve, lunch)                                      
 ASCIIString(thank, you)                                                        
 ASCIIString(i, live, in, denver)                                               
 ASCIIString(what, is, the, least, expensive, flight, from, atlanta, to, denver)
 ASCIIString(is, there, a, round, trip, flight, that, stops, in, pittsburgh)    
 ASCIIString(round, trip, please)                                               
 A

In [101]:
ordered_sents[50]

RemoteException(2,CapturedException(BoundsError(Tuple{FunctionalCollections.PersistentList{ASCIIString},Float64}[],(1,)),Any[(:getindex,symbol("array.jl"),272,symbol(""),-1,1),(:order,symbol("In[21]"),13,symbol(""),-1,1),(:anonymous,symbol("In[24]"),5,symbol(""),-1,1),(:anonymous,symbol("multi.jl"),892,symbol(""),-1,1),(:run_work_thunk,symbol("multi.jl"),645,symbol(""),-1,1),(:anonymous,symbol("multi.jl"),892,symbol("task.jl"),59,1)]))

In [103]:
unordered_output[short_cases][eval_cases][50]

10-element Array{ASCIIString,1}:
 "than"      
 "are"       
 "dollars"   
 "what"      
 "fares"     
 "hundred"   
 "the"       
 "less"      
 "**START1**"
 "four"      

In [104]:
true_ordered_sents[50]

9-element Array{SubString{ASCIIString},1}:
 "what"   
 "are"    
 "the"    
 "fares"  
 "less"   
 "than"   
 "four"   
 "hundred"
 "dollars"

In [77]:
#x= ["A", "B", "C", "D"]
#y= UTF8String["A", "B", "C", "D"]
x = true_ordered_sents[10]
y=ordered_sents[10] |> collect
pycall(nltk_bleu.bleu, PyAny, Any[x], y, Any[0.25, 0.25, 0.25, 0.25])

0.6985342056580097

In [66]:
Any[true_ordered_sents[1]]


1-element Array{Any,1}:
 SubString{ASCIIString}["i","want","to","go","from","denver","to","oakland"]

8-element Array{Any,1}:
 "i"      
 "want"   
 "to"     
 "go"     
 "from"   
 "denver" 
 "to"     
 "oakland"

1.0

LoadError: LoadError: PyError (:PyObject_Call) <type 'exceptions.TypeError'>
TypeError("'PyCall.jlwrap' object is not iterable",)
  File "/usr/local/lib/python2.7/dist-packages/nltk/translate/bleu_score.py", line 69, in bleu
    s = math.fsum(w * math.log(p_n) for w, p_n in zip(weights, p_ns))
  File "/usr/local/lib/python2.7/dist-packages/nltk/translate/bleu_score.py", line 65, in <genexpr>
    for i, _ in enumerate(weights, start=1)
  File "/usr/local/lib/python2.7/dist-packages/nltk/translate/bleu_score.py", line 155, in _modified_precision
    counts = Counter(ngrams(hypothesis, n))
  File "/usr/lib/python2.7/collections.py", line 450, in __init__
    self.update(iterable, **kwds)
  File "/usr/lib/python2.7/collections.py", line 531, in update
    for elem in iterable:
  File "/usr/local/lib/python2.7/dist-packages/nltk/util.py", line 411, in ngrams
    sequence = iter(sequence)

while loading In[28], in expression starting on line 1

In [6]:
nltk_bleu._modified_precision(Any[reference1, reference2, reference3],candidate1, 1)

0.9444444444444444

In [ ]:
@pyimport pdb

In [ ]:
pdb.runcall(nltk_bleu._modified_precision, [reference1, reference2, reference3],candidate1, 1)

In [ ]:
# I think I have to reimplement BLEU in julia as for some reason it does not play nice with PyCall
# Can basically port http://www.nltk.org/_modules/nltk/align/bleu_score.html#bleu

In [ ]:
pycall(nltk_bleu.bleu, Int, candidate1, [reference1], weights)

In [ ]:
@pyimport nltk.util as nltk_util
nltk_util.ngrams(candidate1,2) |> py_collections.Counter

In [ ]:
@pyimport collections as py_collections

In [ ]:
@pyimport nltk.util as nltk_util
ngs = nltk_util.ngrams(candidate1,2)
pycall(py_collections.Counter, PyObject, ngs)

In [ ]:
import nltk